# Output tags in IOB format for NER analysis
### Make function that takes a vocab list and a set of MWOs and returns MWOs in bio format
#### TODO: Handle SI, PI

#### https://pythonprogramming.net/using-bio-tags-create-named-entity-lists/

In [26]:
import pandas as pd
from pathlib import Path
import nestor
# from nestor import keyword as kex
# from nestor.datasets as nd

In [27]:
# Get raw MWOs
df = (nd.load_excavators(cleaned=False)  # already formats dates
      .rename(columns={'BscStartDate': 'StartDate'})
      )

# Change date column to DateTime objects
df.head(5)

,StartDate,Asset,OriginalShorttext,PMType,Cost
0,2004-07-01,A,BUCKET WON'T OPEN,PM01,183.05
1,2005-03-20,A,L/H BUCKET CYL LEAKING.,PM01,407.4
2,2006-05-05,A,SWAP BUCKET,PM01,0
3,2006-07-11,A,FIT BUCKET TOOTH,PM01,0
4,2006-11-10,A,REFIT BUCKET TOOTH,PM01,1157.27


In [28]:
# Get tagged MWOs
tags = pd.read_csv(Path.home()/'Documents'/'datasets'/'1g_original.csv')
tags.head(5)

,tokens,NE,alias,notes,score
0,replace,S,replace,NaN,0.033502
1,bucket,I,bucket,NaN,0.018969
2,repair,S,repair,NaN,0.017499
3,grease,I,grease,NaN,0.017377
4,leak,P,leak,NaN,0.016591


In [29]:
# Get vocab file (unigrams)
vocab_1 = kex.generate_vocabulary_df(
      kex.TokenExtractor(),  # doesn't need to be fitted, since vocab exists
      filename=Path.home()/'Documents'/'datasets'/'1g_original.csv'
)
vocab_1.head(5)

No model fitted, but file already exists. Importing...


/Users/amc8/miniconda3/envs/bio_output/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass msg=The tfidf vector is not fitted as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


,NE,alias,notes,score
tokens,,,,
replace,S,replace,NaN,0.033502
bucket,I,bucket,NaN,0.018969
repair,S,repair,NaN,0.017499
grease,I,grease,NaN,0.017377
leak,P,leak,NaN,0.016591


In [30]:
# Get vocab file (bigrams)
vocab_2 = kex.generate_vocabulary_df(
      kex.TokenExtractor(),  # doesn't need to be fitted, since vocab exists
      filename=Path.home()/'Documents'/'datasets'/'2g_original.csv'
)
vocab_2.head(5)

No model fitted, but file already exists. Importing...


/Users/amc8/miniconda3/envs/bio_output/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass msg=The tfidf vector is not fitted as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


,NE,alias,notes,score
tokens,,,,
oil leak,P I,oil leak,NaN,0.012519
grease line,I,grease_line,NaN,0.008324
bucket tooth,I,bucket_tooth,NaN,0.006528
broken grease,P I,broken grease,NaN,0.006921
jump start,S,jump_start,NaN,0.007049


In [31]:
# merge and cleanse NLP-containing columns of the data
nlp_select = kex.NLPSelect(columns=['OriginalShorttext'])
raw_text = nlp_select.transform(df)  # Series
raw_text = raw_text.head(700)

In [32]:
# Use iob_extractor with both 1gram and ngram vocab outputs from Nestor
iob = kex.iob_extractor(raw_text, vocab_1, vocab_df_ngrams=vocab_2)
iob

AttributeError: module 'nestor.keyword' has no attribute 'iob_extractor'